# Demonstrating sklearn Modin Interoperability

### Logistic Regression example taken / adapted from https://www.ritchieng.com/pandas-scikit-learn/

In [1]:
import modin.pandas as pd
import snowflake.snowpark.modin.plugin
import numpy as np
from snowflake.snowpark.session import Session; session = Session.builder.create()


import numpy as np


In [2]:
# From https://www.ritchieng.com/pandas-scikit-learn/

url = 'http://bit.ly/kaggletrain'
train = pd.read_csv(url)

In [4]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S


In [3]:
# Pclass: passenger class
# Parch: parents and children
feature_cols = ['Pclass', 'Parch']

In [5]:
# you want all rows, and the feature_cols' columns
X = train.loc[:, feature_cols]

In [6]:
# now we want to create our response vector
y = train.Survived

In [7]:
# 1. import
from sklearn.linear_model import LogisticRegression

# 2. instantiate model
logreg = LogisticRegression()

# 3. fit 
logreg.fit(X, y)

LogisticRegression()

In [8]:
url_test = 'http://bit.ly/kaggletest'
test = pd.read_csv(url_test)

In [9]:
# missing Survived column because we are predicting
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,None,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,None,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,None,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,None,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,None,S


In [10]:
X_new = test.loc[:, feature_cols]

In [11]:
# 4. predict
new_pred_class = logreg.predict(X_new)

In [12]:
# kaggle wants 2 columns
# new_pred_class
# PassengerId

# pandas would align them next to each other
# to ensure the first column is PassengerId, use .set_index
kaggle_data = pd.DataFrame({'PassengerId':test.PassengerId, 'Survived':new_pred_class}).set_index('PassengerId')
kaggle_data.to_csv('sub.csv')

Please refer to https://modin.readthedocs.io/en/stable/supported_apis/defaulting_to_pandas.html for explanation.


In [13]:
# save train data to disk using pickle
train.to_pickle('train.pkl')

NotImplementedError: Snowpark pandas does not yet support the method DataFrame.to_pickle

In [ ]:
# read data
pd.read_pickle('train.pkl')

In [14]:
# From https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html

import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import Normalizer
ct = ColumnTransformer(
    [("norm1", Normalizer(norm='l1'), [0, 1]),
     ("norm2", Normalizer(norm='l1'), slice(2, 4))])
X = pd.DataFrame(np.array([[0., 1., 2., 2.],
              [1., 1., 0., 1.]]))
# Normalizer scales each row of X to unit norm. A separate scaling
# is applied for the two first and two last elements of each
# row independently.
ct.fit_transform(X)

array([[0. , 1. , 0.5, 0.5],
       [0.5, 0.5, 0. , 1. ]])

In [15]:
from sklearn.feature_extraction import FeatureHasher
from sklearn.preprocessing import MinMaxScaler
X = pd.DataFrame({
    "documents": [["First item"], ["second one here"], ["Is this the last?"]],
    "width": [3, 4, 5],
})  
ct = ColumnTransformer(
    [("text_preprocess", FeatureHasher(input_type="string"), "documents"),
     ("num_preprocess", MinMaxScaler(), ["width"])])
X_trans = ct.fit_transform(X)

In [16]:
# From https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html

import numpy as np
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
imp_mean.fit(pd.DataFrame([[7, 2, 3], [4, np.nan, 6], [10, 5, 9]]))

X = pd.DataFrame([[np.nan, 2, 3], [4, np.nan, 6], [10, np.nan, 9]])
print(imp_mean.transform(X))

[[ 7.   2.   3. ]
 [ 4.   3.5  6. ]
 [10.   3.5  9. ]]


In [17]:
# From https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

import numpy as np
from sklearn.model_selection import train_test_split
X, y = pd.DataFrame(np.arange(10).reshape((5, 2))), pd.Series(range(5))
X
list(y)

[0, 1, 2, 3, 4]

In [18]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

In [19]:
type(X_train)

modin.pandas.dataframe.DataFrame

In [20]:
train_test_split(y, shuffle=False)

[0    0
 1    1
 2    2
 dtype: int64,
 3    3
 4    4
 dtype: int64]

### Linear Regression example taken / adapted from https://github.com/chendaniely/2021-07-13-scipy-pandas/blob/main/05-models.ipynb

In [21]:
import seaborn as sns

In [30]:
tips = sns.load_dataset("tips").astype('string')
tips = pd.DataFrame(tips)

In [31]:
pd.get_dummies(tips, drop_first=True)

NotImplementedError: get_dummies with non-default dummy_na, drop_first, and dtype parameters is not supported yet in Snowpark pandas.

In [32]:
from sklearn import linear_model

In [33]:
# 1. create the model object
lr = linear_model.LinearRegression()

In [34]:
# 2. fit the model object
lr.fit(X=tips[["total_bill", "size"]], y=tips["tip"])

LinearRegression()

In [35]:
# look at the coefficients
lr.coef_

array([0.09271334, 0.19259779])

In [36]:
# look at the intercept
lr.intercept_

0.6689447408125031

In [37]:
tips_dummy = pd.get_dummies(tips, drop_first=True)[["tip", "total_bill", "smoker_No"]]
tips_dummy

NotImplementedError: get_dummies with non-default dummy_na, drop_first, and dtype parameters is not supported yet in Snowpark pandas.

In [38]:
lr2 = linear_model.LinearRegression()
lr2.fit(X=tips_dummy.iloc[:, 1:], y=tips_dummy["tip"])

NameError: name 'tips_dummy' is not defined

In [39]:
lr2.coef_, lr2.intercept_

AttributeError: 'LinearRegression' object has no attribute 'coef_'

In [40]:
new_data = tips_dummy[["total_bill", "smoker_No"]].tail() # not really new data
new_data

NameError: name 'tips_dummy' is not defined

In [41]:
# use the model to give predicted tip values
new_data["predicted_tips"] = lr2.predict(new_data)

NameError: name 'new_data' is not defined

In [ ]:
new_data

In [ ]:
type(new_data)